# QUBO

Quadratic Unconstrained Binary Optimization is a common shape our problem could take so that

\begin{equation}
\begin{split}
\min_{x_{i}\in{\{0,1\}}} \;\; & x^TQx
\end{split}
\end{equation}

This shape is particularly convenient for binary optimization problems as it helps encode pur problem into quantum annealers where these formulation (and its complementary Ising model) are needed. It requires for us to remove the contraint and this is done by adding it to the objective function in the following form

\begin{equation}
\begin{split}
\displaystyle \min_{x_{i}\in{\{0,1\}}} - \sum_{i=1}^{n} x_{i}e_i + \;\theta_1 \sum_{i,j=1}^{n}x_{i}x_{j}c_{ij} +\;&\theta_{2}\left(\sum_{i=1}^{n}x_{i}b_{i}-B\right)^{2}
\end{split}
\end{equation}


The parameters $0\leq\theta_{1},\theta_{2}<\infty$ represent the relative importance of each term to the decision maker, and she is free to change these parameters to best reflect that (we'll give some examples of this in the next paragraph).  The first term in the objective function represents the expected return, i.e. the gain.  The second term represents the variance in the return, i.e. the risk.  Finally, the last term penalizes our decision maker when the sum of all $b_i$ is lower than the total available budget $B$.

Following we will procide to construct that shape with our existing dataset.

In [1]:
import json
import numpy as np

data = None
with open("../data/portfolio-binance-data.json", "r") as jsonfile:
    data = json.load(jsonfile)

returns = data['mu']
covar = data['sigma']

assets = []
costs = []
for row in data['assets']:
    assets.append(row["Asset"])
    costs.append(float(row["Open"]))

# Half the money
budget = np.sum(costs)/0.8

In [2]:
from pyqubo import Array, Placeholder, Constraint

num_assets = len(assets)
x = Array.create('x', shape=num_assets, vartype='BINARY')

# Profit generated by each asset individually
H_linear_profit = 0.0
for i in range(num_assets):
    H_linear_profit += Constraint(
        returns[i] * x[i], label='profit({})'.format(i)
    )

# Risk obtained from the covariance matrix
H_quadratic = 0.0
for i in range(num_assets):
    for j in range(i + 1, num_assets):
        H_quadratic += Constraint(
            covar[i][j] * x[i] * x[j], label='risk({}, {})'.format(i, j)
        )

# Constraint (budget)
H_linear_budget = 0.0
for i in range(num_assets):
    H_linear_budget += Constraint(costs[i]*x[i], label='slot({})'.format(i))

# Final shape of the problem
theta1 = Placeholder('theta1')
theta2 = Placeholder('theta2')
H = - H_linear_profit + theta1 * H_quadratic + theta2 * (H_linear_budget - budget)**2
model = H.compile()

Now we could instantiate our model with different levels of risk and balance between risk and budget restriction.

In [3]:
# Set the Lagrange multipliers
theta1=0.5 
theta2=0.3
feed_dict = {'theta1': theta1, 'theta2' : theta2}

# Transform to QUBO.
qubo, offset = model.to_qubo(feed_dict=feed_dict)
qubo

{('x[0]', 'x[0]'): -0.0034463611560800484,
 ('x[4]', 'x[1]'): 0.00011802050203187451,
 ('x[2]', 'x[1]'): 0.0001334856314643793,
 ('x[1]', 'x[0]'): 0.00022046381934961036,
 ('x[4]', 'x[2]'): 7.729083662229302e-05,
 ('x[3]', 'x[1]'): 0.00017781918946576346,
 ('x[4]', 'x[3]'): 0.00023462844760721872,
 ('x[3]', 'x[3]'): 0.0010471616441001168,
 ('x[2]', 'x[2]'): -0.0027946686712104667,
 ('x[3]', 'x[2]'): 0.00011856905489815865,
 ('x[1]', 'x[1]'): -8.771291915804695e-05,
 ('x[3]', 'x[0]'): 0.0001262366943724494,
 ('x[2]', 'x[0]'): 0.0006677990164085108,
 ('x[4]', 'x[4]'): 0.0010577103414685352,
 ('x[4]', 'x[0]'): -1.5181177643823699e-05}

This is the matrix $Q$ that was cited at the begining of the notebook and thanks to this we can implement this problem on a quantum annealer.

# D-Wave

In order to enable connectivity with D-Wave's hardware we will need to save locally the connection token provided for D-Wave's [Leap](https://cloud.dwavesys.com/leap/login/?next=/leap/) platform. After registering follow the instructions [here](https://docs.ocean.dwavesys.com/en/stable/overview/sapi.html) to enable the connection to cloud samplers.

In [5]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
import minorminer
import dimod

# Instanciate Sampler
dwave_sampler = DWaveSampler()

Once the sampler is instantiated, we should know not all nodes of the chip that will encode our variables are connected between one another so an embedding must be fount that makes our problem variables fit into D-Wave's sampler architecture.

In [ ]:
# Construct a problem
bqm = dimod.BinaryQuadraticModel(qubo, dimod.BINARY)

# Get the edge list
target_edgelist = dwave_sampler.edgelist

# And source edge list on the BQM quadratic model
source_edgelist = list(bqm.quadratic)

# Find the embeding
embedding = minorminer.find_embedding(source_edgelist, target_edgelist)
sampler = FixedEmbeddingComposite(dwave_sampler, embedding)

Once that is done, simply as the annealer to anneal the problem and look for the sample with minum energy. We will select 10 microseconds for the annealing time and will do this 10 times as quantum computers may fail finding the best solution every time that are called, so we will need to run it several times. From there, the best solution should rise with higher succes probability.

In [ ]:
ta = 10 # microseconds
num_reads = 10
response = sampler.sample_qubo(qubo, num_reads=num_reads, annealing_time=ta)
best_sample = response.first

In [ ]:
best_sample

In [ ]:
# Selected assets.
solution = [
    int(item[2:-1])  # As each variable has the shape 'x[i]', we will just keep 'i'.
    for item, item_selected in best_sample.sample.items()
    if item_selected
]
print('Selected assets are:')
for i in solution:
    print(assets[i])